※ このNotebookでは、perftool動作確認のための環境構築を
を実施します。


## ssh公開鍵の作成

Jupyter Notebook サーバからVCノードに ssh 接続するために、ssh キーペアを作成しておきます。

In [3]:
# 動作確認用SSHキーペア作成
!mkdir -p ~/.ssh
!test -f ~/.ssh/id_rsa || ssh-keygen -t rsa -f ~/.ssh/id_rsa -N ""

# VCノードの作成

## VCP SDK の初期化

VCP SDK を使用ために、はじめにアクセストークンを使用して VCP SDKを初期化してください。  
次のセルを実行すると、VCコントローラのアクセストークンの入力枠が表示されます。  
アクセストークンの値を入力し、Enterキーを押すことで初期化が完了します。

In [1]:
from getpass import getpass
vcc_access_token = getpass()

from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

# VCP SDKの初期化
vcp = VcpSDK(
    vcc_access_token,  # VC Controllerのアクセストークン
)

vcp.version()


········

vcplib:
  filename: /notebooks/notebook/vcpsdk/vcplib/occtr.py
  version: 20.08.0+20200831

vcpsdk:
  filename: /notebooks/notebook/vcpsdk/vcpsdk/vcpsdk.py
  version: 20.10.0+2021001

  plugin:
    aws: 1.2+20191001
    aws_disk: 1.0+20190408
    aws_spot: 1.1+20191001
    azure: 1.2+20191001
    vmware: 1.1+20191001
    azure_disk: 1.0+20190408
    sakura: 1.1+20191001
    sakura_disk: 1.0+20190930
    oracle: 1.0+20200331
    oracle_disk: 1.0+20200331
    aic: 1.2+20191001
    abc: 1.3+20190408
    hokudai: 1.1+20191001
    chameleon: 1.0+20200831
    chameleon_ext: 20200831
    gcp: 1.0+20190408
    onpremises: 1.0+20190408

vc_controller:
  host: 10.0.0.1
  name: vcc1045
  wait_timeout_sec: 1000(default 15min)
  vc_controller: 20.10.1+20210101
  vc_controller_git_tag: 20.10.1

  plugin:
    vmware: 1.1+20210101
    chameleon: 1.0+20210101
    azure: 1.2+20210101
    aws_spot: 1.1+20210101
    aic: 1.2+20210101
    sakura: 1.1+20210101
    hokudai: 1.1+20210101
    abc: 1.

### もしエラーが発生したら...

- `Exception: ('VCP', 'config vc failed')` ... 使われたアクセストークンが正しくない
- `Exception: ('VCP', 'server error')` ... VCコントローラのIPアドレス設定が正しくないか、ネットワーク接続エラーが発生している

## VCノードの起動

次のセルを実行し、動作確認用のVCノードを1個作成します。  
クラウドのインスタンスを起動してVCノード用の初期設定を行うため、実行完了まで数分程度を要します。

In [9]:
# UnitGroupの作成
unit_group = vcp.create_ugroup(
    'sinetstream01'    # UnitGroupの名前
)

spec = vcp.get_spec('aws', 'small')
spec.set_ssh_pubkey('/home/jovyan/.ssh/id_rsa.pub')
spec.num_nodes = 2

unit = unit_group.create_unit(
    'aws-testnodes',    # Unit名の指定
    spec
)


2021-03-18 14:56:22,623 - INFO - BOOTING ... 0 sec
2021-03-18 14:56:27,750 - INFO - BOOTING ... 5 sec
2021-03-18 14:56:32,873 - INFO - BOOTING ... 10 sec
2021-03-18 14:56:38,006 - INFO - BOOTING ... 15 sec
2021-03-18 14:56:43,135 - INFO - BOOTING ... 20 sec
2021-03-18 14:56:48,265 - INFO - BOOTING ... 25 sec
2021-03-18 14:56:53,394 - INFO - BOOTING ... 30 sec
2021-03-18 14:56:58,531 - INFO - BOOTING ... 35 sec
2021-03-18 14:57:03,667 - INFO - BOOTING ... 40 sec
2021-03-18 14:57:08,801 - INFO - BOOTING ... 45 sec
2021-03-18 14:57:13,930 - INFO - BOOTING ... 50 sec
2021-03-18 14:57:19,086 - INFO - BOOTING ... 55 sec
2021-03-18 14:57:24,443 - INFO - BOOTING ... 60 sec
2021-03-18 14:57:29,707 - INFO - BOOTING ... 65 sec
2021-03-18 14:57:34,902 - INFO - BOOTING ... 70 sec
2021-03-18 14:57:40,039 - INFO - BOOTING ... 75 sec
2021-03-18 14:57:45,172 - INFO - BOOTING ... 80 sec
2021-03-18 14:57:50,310 - INFO - BOOTING ... 85 sec
2021-03-18 14:57:55,440 - INFO - BOOTING ... 90 sec
2021-03-18 14:

### もしエラーが発生したら...

AWSのクレデンシャル情報が正しくない場合、以下のようなエラーが発生します。  
Valut に設定した内容を確認してください。

```
*** HAS ERROR:
---
2019/06/12 05:09:13 UTC: * provider.aws: InvalidClientTokenId: The security token included in the request is invalid.
---
```

## 起動したVCノードの確認

次のセルを実行し、起動したVCノードを確認してください。

In [10]:
unit.df_nodes()

,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,99,sinetstream01,aws-testnodes,RUNNING,1,57e0c0fc...,RUNNING,172.30.2.7,i-098a811c793e9520e,VCP-f7944417-b6973f9d,none
1,99,sinetstream01,aws-testnodes,RUNNING,2,a87b6a07...,RUNNING,172.30.2.36,i-03a60dba52e253e2f,VCP-f7944417-b6973f9d,none


## VCノードへのsshログイン

JupyterNotebook の Terminal 機能を用いて、作成したVCノードにsshログインすることができます。  
VCノードのIPアドレスは上記の `unit.df_nodes()` の出力に含まれています。

```
$ ssh -i ~/.ssh/id_rsa root@{cloud_instance_address}
```

In [11]:
# unit_group.find_ip_addresses() は UnitGroup内の全VCノードのIPアドレスのリストを返します
ip_addresses = unit_group.find_ip_addresses(node_state='RUNNING')
client_ip = ip_addresses[0]
server_ip = ip_addresses[1]

まず、ログインする前に ~/.ssh/known_hosts のホストキーを更新します。

In [12]:
!touch ~/.ssh/known_hosts
# ~/.ssh/known_hosts から古いホストキーを削除する
!ssh-keygen -R {client_ip}
!ssh-keygen -R {server_ip}

# ホストキーの登録
!ssh-keyscan -H {client_ip} >> ~/.ssh/known_hosts
!ssh-keyscan -H {server_ip} >> ~/.ssh/known_hosts

# Host 172.30.2.7 found: line 16
# Host 172.30.2.7 found: line 17
# Host 172.30.2.7 found: line 18
/home/jovyan/.ssh/known_hosts updated.
Original contents retained as /home/jovyan/.ssh/known_hosts.old
Host 172.30.2.36 not found in /home/jovyan/.ssh/known_hosts
# 172.30.2.7:22 SSH-2.0-OpenSSH_8.1
# 172.30.2.7:22 SSH-2.0-OpenSSH_8.1
# 172.30.2.7:22 SSH-2.0-OpenSSH_8.1
# 172.30.2.36:22 SSH-2.0-OpenSSH_8.1
# 172.30.2.36:22 SSH-2.0-OpenSSH_8.1
# 172.30.2.36:22 SSH-2.0-OpenSSH_8.1


In [15]:
import os
ssh_private_key = os.path.expanduser('~/.ssh/id_rsa')
ssh_opts = f"-i {ssh_private_key} -l root"

In [16]:
!ssh {ssh_opts} {client_ip} uname -a
!ssh {ssh_opts} {server_ip} uname -a

Linux ip-172-30-2-7 4.4.0-159-generic #187-Ubuntu SMP Thu Aug 1 16:28:06 UTC 2019 x86_64 Linux
Linux ip-172-30-2-36 4.4.0-159-generic #187-Ubuntu SMP Thu Aug 1 16:28:06 UTC 2019 x86_64 Linux


##  Brokerを準備する

In [18]:
!ssh {ssh_opts} {server_ip} /usr/local/bin/docker run -d --name broker --hostname broker -p 1883:1883 -p 9092:9092 sinetstream/tutorial:1.0.0

Unable to find image 'sinetstream/tutorial:1.0.0' locally
1.0.0: Pulling from sinetstream/tutorial
f34b00c7da20: Pulling fs layer
51958a606945: Pulling fs layer
84bd5f3a8eec: Pulling fs layer
c48210ad5f8a: Pulling fs layer
5178c184ad75: Pulling fs layer
7b78a55c12d0: Pulling fs layer
00e788353df9: Pulling fs layer
834ff8bc3f6e: Pulling fs layer
487060dd475d: Pulling fs layer
7b78a55c12d0: Waiting
00e788353df9: Waiting
834ff8bc3f6e: Waiting
487060dd475d: Waiting
5178c184ad75: Waiting
c48210ad5f8a: Waiting
f34b00c7da20: Verifying Checksum
f34b00c7da20: Download complete
84bd5f3a8eec: Verifying Checksum
84bd5f3a8eec: Download complete
c48210ad5f8a: Verifying Checksum
c48210ad5f8a: Download complete
f34b00c7da20: Pull complete
5178c184ad75: Verifying Checksum
5178c184ad75: Download complete
00e788353df9: Verifying Checksum
00e788353df9: Download complete
51958a606945: Verifying Checksum
51958a606945: Download complete
7b78a55c12d0: Verifying Checksum
7b78a55c12d0: Download complete
487060d

In [20]:
!ssh {ssh_opts} {server_ip} /usr/local/bin/docker ps -l
!ssh {ssh_opts} {server_ip} /usr/local/bin/docker exec -t broker ps ax

CONTAINER ID        IMAGE                        COMMAND                  CREATED             STATUS              PORTS                                            NAMES
e4d92eb5ef85        sinetstream/tutorial:1.0.0   "/usr/local/bin/supe…"   2 minutes ago       Up 2 minutes        0.0.0.0:1883->1883/tcp, 0.0.0.0:9092->9092/tcp   broker
  PID TTY      STAT   TIME COMMAND
    1 ?        Ss     0:00 /usr/bin/python3 /usr/local/bin/supervisord -n -c /et
    9 ?        Sl     0:06 java -Xmx1G -Xms1G -server -XX:+UseG1GC -XX:MaxGCPaus
   10 ?        S      0:00 /usr/sbin/mosquitto -c /etc/mosquitto/mosquitto.conf
   12 ?        Sl     0:01 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGC
  717 pts/0    Rs+    0:00 ps ax


## pertool実行環境を準備する

In [22]:
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker run -d --name reader --hostname reader -e ENABLE_BROKER=false --add-host=broker:{server_ip} sinetstream/tutorial:1.0.0

Unable to find image 'sinetstream/tutorial:1.0.0' locally
1.0.0: Pulling from sinetstream/tutorial
f34b00c7da20: Pulling fs layer
51958a606945: Pulling fs layer
84bd5f3a8eec: Pulling fs layer
c48210ad5f8a: Pulling fs layer
5178c184ad75: Pulling fs layer
7b78a55c12d0: Pulling fs layer
00e788353df9: Pulling fs layer
834ff8bc3f6e: Pulling fs layer
487060dd475d: Pulling fs layer
c48210ad5f8a: Waiting
5178c184ad75: Waiting
7b78a55c12d0: Waiting
00e788353df9: Waiting
834ff8bc3f6e: Waiting
487060dd475d: Waiting
84bd5f3a8eec: Verifying Checksum
84bd5f3a8eec: Download complete
c48210ad5f8a: Verifying Checksum
c48210ad5f8a: Download complete
5178c184ad75: Verifying Checksum
5178c184ad75: Download complete
f34b00c7da20: Verifying Checksum
f34b00c7da20: Download complete
00e788353df9: Verifying Checksum
00e788353df9: Download complete
7b78a55c12d0: Verifying Checksum
7b78a55c12d0: Download complete
834ff8bc3f6e: Verifying Checksum
834ff8bc3f6e: Download complete
487060dd475d: Verifying Checksum
48

In [24]:
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker ps -l
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t reader ps ax

CONTAINER ID        IMAGE                        COMMAND                  CREATED             STATUS              PORTS                NAMES
4e81e87190e4        sinetstream/tutorial:1.0.0   "/usr/local/bin/supe…"   3 minutes ago       Up 3 minutes        1883/tcp, 9092/tcp   reader
  PID TTY      STAT   TIME COMMAND
    1 ?        Ss     0:00 /usr/bin/python3 /usr/local/bin/supervisord -n -c /et
   30 pts/0    Rs+    0:00 ps ax


Reader用のコンテナを起動する際に指定した --add-host は　Readerコンテナの /etc/hosts に、Broker の IPアドレスを登録するためのものです。 Kafkaブローカーを利用するためにはサーバアドレスの名前解決が必要となるため、このパラメータの指定を追加しています。 Readerコンテナの /etc/hosts を表示して Broker のIPアドレスが登録されていることを確認します。

In [25]:
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t reader cat /etc/hosts

### SINETStreamをインストールする

In [26]:
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader pip3 install --user sinetstream-kafka sinetstream-mqtt

     |################################| 256kB 12.4MB/s eta 0:00:01
     |################################| 102kB 9.5MB/s eta 0:00:01
     |################################| 1.9MB 14.8MB/s eta 0:00:01
    Running setup.py install for promise ... done
    Running setup.py install for avro-python3 ... done
    Running setup.py install for paho-mqtt ... done
You should consider upgrading via the 'pip install --upgrade pip' command.


最後に Successfully installed ...と表示されていれば、ライブラリのインストールに成功しています。 確認のためインストールされている Python3　ライブラリの一覧を表示してみます。

In [27]:
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader pip3 list

Package           Version
----------------- -------
avro-python3      1.10.2 
kafka-python      2.0.2  
paho-mqtt         1.5.1  
pip               19.3.1 
promise           2.3    
pycryptodomex     3.10.1 
PyYAML            3.12   
setuptools        42.0.2 
sinetstream       1.5.0  
sinetstream-kafka 1.5.0  
sinetstream-mqtt  1.5.0  
six               1.15.0 
supervisor        4.1.0  
You should consider upgrading via the 'pip install --upgrade pip' command.


# SINETStreamの動作確認

## Readerのプログラムと設定ファイルを準備する

In [32]:
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader mkdir -p /home/user01/sinetstream/reader

In [41]:
ss_url='https://raw.githubusercontent.com/nii-gakunin-cloud/sinetstream/master'

!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader curl -O {ss_url}/docs/tutorial/.sinetstream_config.yml -O {ss_url}/python/sample/text/consumer.py
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader mv .sinetstream_config.yml sinetstream/reader/
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader mv consumer.py sinetstream/reader/


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   239  100   239    0     0    540      0 --:--:-- --:--:-- --:--:--   540
100  1687  100  1687    0     0   6555      0 --:--:-- --:--:-- --:--:--  6564


In [42]:
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader ls -a /home/user01/sinetstream/reader

## Writerのプログラムと設定ファイルを準備する

In [43]:
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader mkdir -p /home/user01/sinetstream/writer

In [44]:
ss_url='https://raw.githubusercontent.com/nii-gakunin-cloud/sinetstream/master'

!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader curl -O {ss_url}/docs/tutorial/.sinetstream_config.yml -O {ss_url}/python/sample/text/producer.py
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader mv .sinetstream_config.yml sinetstream/writer/
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader mv producer.py sinetstream/writer/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   239  100   239    0     0    501      0 --:--:-- --:--:-- --:--:--   502
100  1608  100  1608    0     0   5607      0 --:--:-- --:--:-- --:--:--  523k


In [45]:
!ssh {ssh_opts} {client_ip} /usr/local/bin/docker exec -t -u user01 reader ls -a /home/user01/sinetstream/writer

https://www.sinetstream.net/docs/tutorial/TUTORIAL-STEP2.html 3.1. Kafkaブローカーとの間でメッセージの送受信を行う と同様に実行してみる

# VCノードの削除
次のセルを実行し、動作確認に使用したクラウドインスタンスを削除してください。  
実行完了まで数分程度を要します。

In [8]:
unit_group.cleanup()

2021-03-18 14:56:14,734 - INFO - cleanup completed. vc sinetstream01 is cleanup(no unit)
